In [3]:
import pandas as pd

import tensorflow as tf

import numpy as np



training_epochs = 50

learning_rate = 0.003


test_s = pd.read_csv("test_p", header=None)


def model(col_size):
    
    X = tf.placeholder(tf.float32, [None,col_size])
    Y = tf.placeholder(tf.float32, [None, 2])

    W1 = tf.Variable(tf.truncated_normal([col_size,50],stddev=0.1))
    W2 = tf.Variable(tf.truncated_normal([50,2],stddev=0.1))
    B1 = tf.Variable(tf.zeros([50]))
    B2 = tf.Variable(tf.zeros([2]))
    
    
    Y1 = tf.nn.softmax(tf.add(tf.matmul(X,W1),B1))
    predict = tf.nn.softmax(tf.add(tf.matmul(Y1,W2),B2))

    cost = -tf.reduce_sum(Y*tf.log(predict))

    return X,Y,predict,cost


x, y , predict , cost = model(len(test_s.columns)-2)

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)


init = tf.global_variables_initializer()


with tf.Session() as sess:

    sess.run(init)

    print "Start Training"

    epoch = 0
    avg_cost = float('inf')
    
    while epoch <= training_epochs and avg_cost >= 10:
        avg_cost = 0.
        total_batch = 0
       # Loop over all batches
        for chunk in pd.read_csv("train_p", header=None, chunksize = 500):
            total_batch += 1
            batch_xs = chunk[chunk.columns[:-2]]
            batch_ys = chunk[chunk.columns[-2:]]
            
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                         y: batch_ys})

            avg_cost += c
        avg_cost = avg_cost / total_batch
        epoch += 1
        if (epoch+1) % 10 == 0:
            s = "Epoch:" + str(epoch+1) + " " +  "cost=" + str(avg_cost) + "\n"
            print s
        
    print "Done Training"

    print "Testing"
    # Test model
    test_xs = test_s[test_s.columns[:-2]]
    test_ys = test_s[test_s.columns[-2:]]
    
    correct_prediction = tf.equal(tf.argmax(predict,1),tf.argmax(y,1))
    
    y_p = tf.argmax(predict,1)

    y_pred = sess.run(y_p,feed_dict={x:test_xs, y:test_ys})
    
    #Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))
    print("Accuracy:" + str(accuracy.eval({x:test_xs, y: test_ys})) + "\n")
    


    print "Done Testing"


print "Program end"





Start Training
Epoch:10 cost=244.997216055

Epoch:20 cost=230.44006983

Epoch:30 cost=224.441392238

Epoch:40 cost=221.876109817

Epoch:50 cost=220.167687639

Done Training
Testing
Accuracy:0.7835

Done Testing
Program end
